# Monte Carlo Scheduling — 00: Intro & Problem Setup

**Projet :** Optimisation d’ordonnancement (Scheduling) par méthodes de Monte Carlo  
**Objectif :** développer un prototype capable de générer, simuler, évaluer et améliorer des plannings pour un ensemble de tâches sur plusieurs ressources, afin de minimiser le temps total de production et, optionnellement, les retards et d’autres critères.

> Ce notebook introduit le problème, la modélisation, les métriques et la logique Monte Carlo.


## Plan du projet et organisation

Le dépôt est structuré en notebooks progressifs :

- **00** — Intro, modélisation, métriques, vue d’ensemble des méthodes Monte Carlo
- **01** — Génération d’instances (données synthétiques réalistes) + export
- **02** — Simulateur + calcul de métriques (makespan, tardiness, score)
- **03** — Baselines / heuristiques classiques (références)
- **04** — Monte Carlo Random Search (échantillonnage massif)
- **05** — Monte Carlo amélioré : Recuit simulé (Simulated Annealing)
- **06** — Expériences & comparaison reproductibles
- **07** — Démo finale (pipeline complet + figures)

Le code réutilisable sera placé dans `src/` et les données dans `data/`.


## 1) Problème d’ordonnancement (Scheduling)

On considère :

- un ensemble de tâches (jobs) : \(J = \{0,1,\dots,n-1\}\)
- un ensemble de machines / ressources : \(M = \{0,1,\dots,m-1\}\)

Chaque tâche doit être exécutée **une seule fois** sur **une seule machine**.  
Chaque machine exécute au plus **une tâche à la fois**.

Nous retenons le cas **machines parallèles non identiques** (dit “unrelated parallel machines”) :
- temps de traitement \(p_{j,k}\) = durée de la tâche \(j\) sur la machine \(k\)

Ce modèle est général : le cas “machines identiques” est un sous-cas (\(p_{j,k} = p_j\)).


## 2) Données et variables

### Données
- $p_{j,k} \ge 0$ : durée de la tâche $j$ sur la machine $k$
- (option) $d_j$ : deadline (date limite)
- (option) $w_j$ : poids / priorité
- (option) $r_j$ : release time (date de disponibilité)

### Décisions (un planning)

Un planning peut être représenté comme une famille de séquences :

$$
S = (\pi_0, \pi_1, \dots, \pi_{m-1})
$$

où $\pi_k$ est la liste ordonnée des tâches exécutées sur la machine $k$.

### Contraintes
- chaque tâche apparaît **exactement une fois** dans l’union des $\pi_k$
- sur une machine, l’exécution est **séquentielle** (sans chevauchement)
- pas de **préemption** (une tâche ne s’interrompt pas)


## 3) Évaluation par simulation

Pour comparer deux plannings, on les **simule** afin de calculer :
- $S_j$ : temps de début de la tâche $j$
- $C_j$ : temps de fin de la tâche $j$

Sur une machine $k$, si la séquence est :

$$
\pi_k = (j_1, j_2, \dots)
$$

on a typiquement :

- $S_{j_1} = \max(0, r_{j_1})$
- $S_{j_t} = \max(C_{j_{t-1}}, r_{j_t}) \quad \text{pour } t \ge 2$

et :

$$
C_{j_t} = S_{j_t} + p_{j_t,k}
$$

La simulation permet ainsi de reconstruire l’exécution complète du planning et de calculer des **métriques globales** telles que le makespan, le retard total ou le retard pondéré.


## 4) Métriques d’optimisation

### Makespan (objectif principal)

$$
C_{\max} = \max_{j \in J} C_j
$$

Minimiser $C_{\max}$ revient à minimiser la **durée totale de production**, c’est-à-dire le temps de fin de la dernière tâche.

### Tardiness (retard) — optionnel

$$
T_j = \max(0, C_j - d_j)
$$

À partir de cette définition, on peut considérer :
- **Retard total** :  
  $$
  \sum_j T_j
  $$
- **Retard pondéré** (si des priorités sont définies) :  
  $$
  \sum_j w_j T_j
  $$


## 5) Fonction objectif (score unique)

Pour comparer facilement des solutions dans une boucle Monte Carlo, on utilise un score scalaire :

$$
\text{score} = \alpha C_{\max} + \beta \sum_j T_j + \gamma \sum_j w_j T_j
$$

- \(\alpha, \beta, \gamma\) sont des poids choisis selon l’importance des critères.
- Dans la suite, on prendra typiquement \(\alpha = 1\) et \(\beta, \gamma\) plus petits.

> Cette agrégation est standard en optimisation multi-critères lorsqu’on veut une comparaison simple entre solutions.


## 6) Pourquoi des méthodes de Monte Carlo ?

L’espace des plannings possibles est **combinatoire** et croît très rapidement avec \(n\) et \(m\).  
Le problème est généralement **NP-difficile**, ce qui rend l’optimisation exacte impraticable à grande échelle.

Les méthodes de Monte Carlo consistent à :
1. générer des solutions candidates (souvent aléatoires)
2. les évaluer par simulation
3. conserver/améliorer les meilleures solutions

Elles sont adaptées aux problèmes où :
- l’évaluation d’une solution est relativement simple (simulation)
- l’espace de recherche est très grand
- on vise de bonnes solutions approchées plutôt qu’un optimum exact


## 7) Méthodes Monte Carlo implémentées

### (A) Monte Carlo Random Search (baseline Monte Carlo)
- on échantillonne \(K\) plannings aléatoires
- on garde le meilleur

### (B) Monte Carlo amélioré : Recuit simulé (Simulated Annealing)
- on part d’un planning initial
- on applique des **mouvements locaux** (swap, move, swap inter-machines…)
- on accepte toujours les améliorations
- on accepte parfois des dégradations avec une probabilité \(\exp(-\Delta / T)\) pour échapper aux minima locaux

Ces deux approches permettent :
- d’observer une **courbe de convergence**
- de comparer à des heuristiques classiques (SPT, EDD, greedy…)


## 8) Livrables et résultats attendus

- **Code** : générateur d’instances, simulateur, baselines, algorithmes Monte Carlo
- **Résultats** :
  - courbes de convergence (best-so-far vs itérations)
  - comparaison (baseline vs Monte Carlo)
  - diagramme de Gantt du meilleur planning
- **Rapport (2–4 pages recommandé)** :
  - formulation du problème
  - description des algorithmes
  - protocole expérimental
  - résultats et discussion


## 9) Roadmap (prochaine étape)

Dans le prochain notebook (**01_Instance_Generation**), on va :
- définir un générateur d’instances réaliste (\(p_{j,k}\), deadlines…)
- sauvegarder les instances dans `data/instances/` (format JSON)
- assurer la reproductibilité (seed)

Ensuite (**02_Simulator_and_Metrics**) :
- on valide la simulation sur un mini-exemple
- on calcule les métriques et le score
